In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

In [2]:
data_path = Path('../input/h-and-m-personalized-fashion-recommendations/')
N = 12

In [3]:
df = pd.read_csv(data_path / 'transactions_train.csv',
                    usecols = ['t_dat', 'customer_id', 'article_id'],
                    dtype={'article_id': str})

df['t_dat'] = pd.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()

In [4]:
df['ldbw'] = df['t_dat'].progress_apply(lambda d: last_ts - (last_ts - d).floor('7D'))

100%|██████████| 31788324/31788324 [22:03<00:00, 24020.71it/s] 


In [5]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

In [6]:
df = df.join(weekly_sales, on=['ldbw', 'article_id'])

In [7]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')
last_day = last_ts.strftime('%Y-%m-%d')

df = df.join(
    weekly_sales.loc[weekly_sales['ldbw']==last_day, ['count']],
    on='article_id', rsuffix="_targ")

df['count_targ'].fillna(0, inplace=True)
del weekly_sales

In [8]:
df['quotient'] = df['count_targ'] / df['count']

In [9]:
target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(N).index.tolist()
del target_sales

In [10]:
purchase_dict = {}

for i in tqdm(df.index):
    cust_id = df.at[i, 'customer_id']
    art_id = df.at[i, 'article_id']
    t_dat = df.at[i, 't_dat']

    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}

    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    x = max(1, (last_ts - t_dat).days)

    a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
    y = a / np.sqrt(x) + b * np.exp(-c*x) - d

    value = df.at[i, 'quotient'] * max(0, y)
    purchase_dict[cust_id][art_id] += value

100%|██████████| 31788324/31788324 [11:28<00:00, 46172.83it/s]


In [11]:
sub = pd.read_csv(data_path / 'sample_submission.csv')

pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in purchase_dict:
        series = pd.Series(purchase_dict[cust_id])
        series = series[series > 0]
        l = series.nlargest(N).index.tolist()
        if len(l) < N:
            l = l + general_pred[:(N-len(l))]
    else:
        l = general_pred
    pred_list.append(' '.join(l))

sub['prediction'] = pred_list
sub.to_csv('submission.csv', index=None)

100%|██████████| 1371980/1371980 [11:08<00:00, 2052.52it/s]
